<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Импорты

In [8]:
#imports
import numpy as np
import re
from sklearn import metrics
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import json
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

Загружу из колаба чтобы долго не качать 1.5 гб

In [3]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Colab Notebooks/DLS_Part2/01.Введение в NLP/HW_01/SO_vectors_200.bin download=1", binary=True)

Было до

In [ ]:
# !wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-04-14 09:40:54--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G   336KB/s    in 95m 46s 

2023-04-14 11:16:43 (247 KB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [ ]:
# from gensim.models.keyedvectors import KeyedVectors
# wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

NameError: ignored

In [ ]:
wv_embeddings['dog']

array([ 0.6851772 , -1.2778991 , -0.41913974,  1.3623164 , -3.1675398 ,
        0.09950767,  0.6402681 , -1.1245339 , -0.6699619 , -0.6998852 ,
        0.4936771 , -0.40500194, -3.0706816 , -2.2809966 ,  0.85798043,
        2.7093108 ,  0.3492745 , -0.03494101, -0.22330493,  1.2290467 ,
        1.7755157 , -3.158358  , -0.6515983 ,  0.7224096 ,  2.3193083 ,
       -1.7969862 ,  0.40903398, -2.744604  , -1.7179952 , -0.914309  ,
       -0.75887376, -0.35140672, -0.5182776 , -1.9097351 , -0.8300773 ,
        0.02147918,  1.1783471 ,  0.03169126, -0.3069023 ,  1.6666299 ,
        0.6711357 , -2.1706133 , -0.11800487,  0.22336982, -1.2075394 ,
       -0.86297905, -0.63865614,  1.1733794 ,  0.10022762,  0.7017279 ,
        2.7290728 , -0.4640484 , -2.1719306 , -0.3562852 , -1.8449957 ,
        0.10270727,  1.1125596 , -0.8364318 ,  1.9513408 , -0.97937447,
        1.2650859 ,  0.06809282,  0.6477318 , -0.52431005, -0.6103959 ,
       -2.979829  , -0.7889965 , -0.11004248,  1.7603841 , -1.05

Тут ошибка

AttributeError: The index2word attribute has been replaced by index_to_key since Gensim 4.0.0.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4


In [ ]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

AttributeError: ignored

Поэтому используем новый атрибут

In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key )}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

#### Решение вопроса 1



[Косинусная мера](https://en.wikipedia.org/wiki/Cosine_similarity) исп для определения близости слов

Подсчет [нормы в Numpy](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html#rac1c834adb66-1)

[Frobebius Norm](https://mathworld.wolfram.com/FrobeniusNorm.html)

норма Фромебиуса на питоне

In [ ]:
b1 = [1,2,3]

In [ ]:
def py_norm(b1):
  res = 0
  for i in b1:
    res += i**2
  
  return np.sqrt(res)

In [ ]:
py_norm(b1)

3.7416573867739413

На нампай

In [ ]:
np.linalg.norm(b1)

3.7416573867739413

Сравню с самописный метод на питон и нампай чтобы проверить себя

In [ ]:
# одинаковый резуьтат
assert py_norm(b1) == np.linalg.norm(b1)

Проверю как считается для матрицы

In [ ]:
a1 = np.arange(4)
a1

array([0, 1, 2, 3])

In [ ]:
np.flip(a1)

array([3, 2, 1, 0])

In [ ]:
a2 = np.vstack((a1,np.flip(a1)))
a2

array([[0, 1, 2, 3],
       [3, 2, 1, 0]])

In [ ]:
a2.shape

(2, 4)

In [ ]:
np.linalg.norm(a2,axis=1)

array([3.74165739, 3.74165739])

Самописный метод косинусной близости

In [ ]:
# my cosin similarity
def my_cosin_sim(target, other):

  target_norm = np.linalg.norm(target)
  other_norm = np.linalg.norm(other, axis = 1)
  res = (target @ other.T) / (target_norm * other_norm)

  # return np.flip(np.sort(my_cosin_sim))
  return res

Проверка метода

In [ ]:
my_cosin_sim(a1,a2)

array([1.        , 0.28571429])

In [ ]:
my_cosin_sim(wv_embeddings['dog'], wv_embeddings['cat', 'dog'])

array([0.6852341, 1.       ], dtype=float32)

In [ ]:
my_cosin_sim(wv_embeddings['dog'], wv_embeddings[wv_embeddings.index_to_key])

array([0.07585394, 0.21143065, 0.00233966, ..., 0.16834746, 0.03601661,
       0.05869402], dtype=float32)

Проверю как работает дефолтная функция cosine_similarity





In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
resh_a1 = np.reshape(a1,(1,-1))

In [ ]:
print(resh_a1)
print(a2)

[[0 1 2 3]]
[[0 1 2 3]
 [3 2 1 0]]


In [ ]:
cosine_similarity(resh_a1,a2)

array([[1.        , 0.28571429]])

Сравню свой самописный метод с метотдом из sklearn

In [ ]:
assert (my_cosin_sim(a1,a2) == cosine_similarity(resh_a1,a2)[0]).all()

AssertionError: ignored

Что то не сходится, проверю

In [ ]:
cosine_similarity(resh_a1,a2)[0][1]

0.28571428571428575

In [ ]:
my_cosin_sim(a1,a2)[1]

0.2857142857142857

Я так понял различие в последнем знаке посля запятой, что не критично


---



Нужная нам функция most_similar. Выводит топ_N самых близких слов

In [ ]:
# most_similar
def most_similar(target_word, other_words = 'all', top_n = 5):
  """
  Returns top_n similar words to target_word

  target_word:str

  other_words:list()
    if omitted then all words will be used

  """
  # вытаскиваем эмбеддинг по слову
  target_word_wv = wv_embeddings[target_word]

  if other_words == 'all':
    other_words_wv = wv_embeddings[wv_embeddings.index_to_key] # эмбеддинги всех слов
    d = dict.fromkeys(wv_embeddings.index_to_key) # словарь для удобного вывода
  else:
    other_words_wv = wv_embeddings[other_words] # эмбеддинги заданных слов
    d = dict.fromkeys(other_words) # словарь для удобного вывода

  # считаем косинусную близость функцией что написали выше
  cosin_sim = my_cosin_sim(target_word_wv, other_words_wv)
  
  # присваиваем в словаре каждому слову найденную косинусную близость
  for key, value in zip(d,cosin_sim):
    d[key] = value
  
  # сортируем и осталяем только топ_n слов из словаря
  sorted_d = sorted(d.items(), key=lambda x:x[1], reverse = True) [:top_n]

  return sorted_d

Проверка

In [ ]:
# проверим для 2х слов
most_similar('dog', ['cat', 'dog'])

[('dog', 1.0), ('cat', 0.6852341)]

In [ ]:
# проверим для всех слов в словаре
most_similar('dog')

[('dog', 1.0),
 ('animal', 0.85641795),
 ('dogs', 0.7880867),
 ('mammal', 0.7623805),
 ('cats', 0.7621255)]

Вижу что в топ_5 слов слово 'cat' не входит, поэтому возьму проитерируюсь по сотне слов и выпишу их

In [ ]:
for i, tupl in enumerate(most_similar('dog', top_n=100)):
  if tupl[0] != 'cat':
    print(i, tupl)
  else:
    print(i, tupl)
    break

0 ('dog', 1.0)
1 ('animal', 0.85641795)
2 ('dogs', 0.7880867)
3 ('mammal', 0.7623805)
4 ('cats', 0.7621255)
5 ('animals', 0.7607938)
6 ('feline', 0.7392399)
7 ('bird', 0.73154885)
8 ('animal1', 0.7219216)
9 ('doggy', 0.72133493)
10 ('labrador', 0.72091323)
11 ('canine', 0.7209057)
12 ('meow', 0.7185295)
13 ('cow', 0.70804447)
14 ('dog2', 0.70579106)
15 ('woof', 0.7050612)
16 ('dog1', 0.7038841)
17 ('dog3', 0.70188195)
18 ('penguin', 0.6970292)
19 ('bulldog', 0.6940488)
20 ('mammals', 0.69313896)
21 ('bark', 0.69137985)
22 ('fruit', 0.6892252)
23 ('reptile', 0.689121)
24 ('furry', 0.68634987)
25 ('carnivore', 0.686295)
26 ('cat', 0.6852341)


#### Ответ 1

Слово 'сat' находится на 26 позиции по схожести со словом 'dog'



### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
        
tokenizer = MyTokenizer()

ПРоверю токенайзер

In [ ]:
tt = tokenizer.tokenize('gasdf asdf asdf asdf , ,sdf ,sdf, Пи34 234 3кс@ )$*:')
tt

['gasdf', 'asdf', 'asdf', 'asdf', 'sdf', 'sdf', 'Пи34', '234', '3кс']

In [39]:
def question_to_vec(question, embeddings, tokenizer, dim=200, lower = False, 
                    lemma = False, stemma = False, stop_words = False ):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """

    # lowercase
    if lower:
      question = question.lower()
    
    # токенизирую
    try:
      tokens = tokenizer.tokenize(question)
    except AttributeError:
      tokens = tokenizer(question)
    result = tokens

    if stop_words:
      result = list(set(result) - set(stopWords))
    
    if lemma:
      lemmatized_sentence = []
      for word in result:
        lemmatized_sentence.append(lemmatizer.lemmatize(word))
      result = lemmatized_sentence
    elif stemma:
      stemmed_sentence = []
      for word in result:
        stemmed_sentence.append(stemmer.stem(word))
      result = stemmed_sentence

    # создаю пустую матриру размера [количество слов в предложении, dim]
    sentence = np.empty([len(result),dim], dtype = 'float32')

    # добавляю эмбеддинг в пустую матрицу, попутно обрабатывая исключения
    for i, token in enumerate(result):
      try:
        sentence[i] = embeddings[token]

      except KeyError:
        sentence[i] = 0

    return np.mean(sentence, axis = 0) # нахожу среднее

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

ПРоверка

In [ ]:
question_to_vec('I love neural networks', wv_embeddings, tokenizer)

array([-0.7606706 , -1.2668446 , -0.9640592 , -1.0282726 ,  0.11937475,
        0.05030221,  0.839131  , -0.05288821,  0.4039777 , -0.27854335,
        0.21684763,  0.7734375 , -2.1490996 , -0.9894655 ,  1.6573721 ,
        0.8759004 ,  0.81314635,  1.385042  , -0.74050355,  0.6540709 ,
       -0.12914719,  0.5537926 , -0.587087  , -0.6620902 , -0.4614458 ,
       -0.11904268, -0.46280986,  0.27501124,  1.510315  ,  0.1970604 ,
       -1.1212592 , -1.0400363 , -0.9037453 , -0.76246274,  2.071588  ,
        1.2941785 , -0.14278634, -0.44245854, -0.08558662, -1.4167764 ,
       -1.3412628 , -0.15315288, -0.52965254,  1.8667755 , -0.11589538,
        0.844126  ,  0.19581264,  0.98649347, -0.43250644, -0.24953559,
        1.2573676 ,  0.68422496, -0.524101  ,  1.7814415 , -1.3225667 ,
        2.353354  ,  0.9182403 , -0.7915811 ,  0.5923178 ,  0.03303018,
        0.8717284 , -0.33585173, -0.6862362 ,  0.01126128, -0.64142317,
       -0.763759  , -0.5371587 ,  0.80454034,  0.6661407 , -0.22

Проверка, если нет предобученного вектора для слова

In [ ]:
question_to_vec('I I I I', wv_embeddings, tokenizer)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

#### Ответ 2

In [ ]:
np.around(question_to_vec('I love neural networks', wv_embeddings, tokenizer)[2], 2)

-0.96

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

#### Ответ 3

Пусть у нас имеется один вопрос.
Пусть дубликат попал на позицию 1. 

Тогда DCG@1 = $\frac{1}{\log_2 (1+1)} * 1 = 1$. В любом другом случае эта метрика будет равна нулю.

Hits@47 = 1. И она будет равна 1 вплоть до значения позиции равной 47, и далее будет равна нулю

Значит максимума выражение будет достигать при значениях позиции дубликата от [2,47] и будет равно 1, а минимума будет достаигать при значениях позиции дубликата от [1]U[48,+∞] и равно нулю

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

#### Ответ 4

In [ ]:
np.around((1/np.log2(1+9))*1,1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. 

$dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [9]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    np_dup_ranks = np.array(dup_ranks)
    indic = np_dup_ranks <= k
    indic_sum = indic.sum()
    hits_value = indic_sum / len(dup_ranks)

    return hits_value    

In [10]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    np_dup_ranks = np.array(dup_ranks)
    indic = np_dup_ranks <= k
    logs = 1/np.log2(1+np_dup_ranks)
    multiplication = logs * indic
    multiplication_sum = multiplication.sum()
    dcg_value = multiplication_sum / len(dup_ranks)

    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [16]:
!unzip '/content/drive/MyDrive/Colab Notebooks/DLS_Part2/01.Введение в NLP/HW_01/stackoverflow_similar_questions.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/DLS_Part2/01.Введение в NLP/HW_01/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


In [ ]:
#!unzip stackoverflow_similar_questions.zip

Считайте данные.

In [17]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
      inner_list = [string.rstrip('\t\n') for string in line.split('\t')]
      data.append(inner_list)
    return data

Нам понадобиться только файл validation.

In [18]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [40]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim = 200,
                    lower = False, lemma= False, stemma = False, stop_words = False):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    # сделать векторное предст предложения question
    vectorized_question = question_to_vec(question, embeddings, tokenizer, dim = dim,
                                          lower = lower, lemma = lemma, stemma = stemma,
                                          stop_words = stop_words)

    # векторизовать кандидатов - сделать список из векторных представлений предложений
    vectorized_candidates = []
    for canditate in candidates:
      vectorized_candidates.append(question_to_vec(canditate, embeddings, tokenizer, dim = dim,
                                                  lower = lower, lemma = lemma, stemma = stemma, 
                                                  stop_words = stop_words))

    # решейпаю нампай массив, тк он размерности (N,) а нужна размерность для sklearn
    # cosine_similarity (1,N)
    resh_vectorized_question = np.reshape(vectorized_question,(1,-1))

    # считаем косинусную близость
    cosin_sim = cosine_similarity(resh_vectorized_question, vectorized_candidates)

    # словарь candidates:cosin_sim
    cos_cand_dict = dict(zip(cosin_sim[0], candidates))

    # словарь i:cosin_sim
    i_cos_dict = dict(zip(range(len(candidates)), cosin_sim[0]))

    # сортируем словарь i:cosin_sim
    sorted_i_cos_dict = sorted(i_cos_dict.items(), key=lambda x:x[1], reverse = True)

    output = []
    #итерируемся по отсорт словарю i_cos_dict и формируем новый список отсортированных туплей
    for tuplle in sorted_i_cos_dict:
        output.append((tuplle[0],cos_cand_dict[tuplle[1]]))
    
    # # сортируем
    # sorted_d = sorted(cos_cand_dict.items(), key=lambda x:x[1], reverse = True)

    return output


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print('ranks ', ranks)
        print([r[0] for r in ranks].index(0))
        print()


ranks  [(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?'), (3, 'How to use jQuery AJAX for an outside domain?')]
1

ranks  [(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]
1



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

#### Ответ 5

102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [19]:
train_data = read_corpus('./data/train.tsv')

In [ ]:
train_data

[['converting string to list',
  'Convert Google results object (pure js) to Python object'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP'],
 ['How to insert CookieCollection to CookieContainer?',
  'C# create cookie from string and send it'],
 ['Updating one element of a bound Observable collection',
  'WPF- How to update the changes in list item of a list'],
 ['MongoDB error on find()',
  'Retrieve only the queried element in an object array in MongoDB collection'],
 ['select2 not displaying search results',
  'How to use jQuery AJAX for an outside domain?'],
 ['Using Reduce to merge multiple data frames with passing arguments and without defining function outside the Reduce (syntax)',
  'R - merge a list of data frames into one data frame with missing values by row'],
 ['Adding Prototype to JavaScript Object L

In [ ]:
len(train_data)

1000000

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

Сначала не буду менять параметры модели. Попробую применить озвученные на семинаре способы:

- Дефеолтный токенайзер - split()
- Другой токенайзер nltk
- n-gramms
- nltk + lowercase
- nltk + lowercase + lemmatezation
- nltk + lowercase + stemming
- nltk + lowercase + lemmatezation + stop words
- nltk + lowercase + stemming + stop words

После этого уже попробую перебрать гиперпараметры W2V

### Вспомогательные функции

In [12]:
def words_maker(train_data, tokenizer, lower = False, lemma = False, stemma = False,
                stop_words = False):
  words=[]
  for sentence in train_data:
    skleika = sentence[0] + ' ' + sentence[1]

    if lower:
      skleika = skleika.lower()

    try:
      tokenized_sentence = tokenizer.tokenize(skleika)
    except AttributeError:
      tokenized_sentence = tokenizer(skleika)
    result = tokenized_sentence

    if stop_words:
      result = list(set(result) - set(stopWords))

    if lemma:
      lemmatized_sentence = []
      for word in result:
        lemmatized_sentence.append(lemmatizer.lemmatize(word))
      result = lemmatized_sentence
    elif stemma:
      stemmed_sentence = []
      for word in result:
        stemmed_sentence.append(stemmer.stem(word))
      result = stemmed_sentence

    words.append(result)
  return words

In [14]:
def mean_of_words(words):
  count_words = []
  for word in words:
    count_words.append(len(word))
  mean = 0
  sum=0
  for word in count_words:
    sum+=word
  mean = sum / len(count_words)

  return mean

In [41]:
def wv_ranking(validation_data, embeddings_trained, tokenizer, dim = 200, lower = False,
               lemma = False, stemma = False, stop_words = False, max_validation_examples = 1000 ):
  wv_ranking = []
  for i, line in enumerate(tqdm(validation_data)):
      if i == max_validation_examples:
          break
      q, *ex = line
      ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, dim = dim,
                              lower = lower, lemma = lemma, stemma = stemma, stop_words = stop_words)
      wv_ranking.append([r[0] for r in ranks].index(0) + 1)
  return wv_ranking

In [ ]:
def gridsearch_w2v(vector_size, window, min_count):
  results = dict()
  for vs in vector_size:
    for w in window:
      for mc in min_count:
        embeddings_trained = Word2Vec(words, 
                                    vector_size=vs,                 
                                    min_count=mc,    
                                    window=w).wv
        results[f'embs_w2v_{vs}_{w}_{mc}'] = embeddings_trained
  return results

In [ ]:
def DCG_Hits_for_gridsearch(gs_ranks):
  for key,value in gs_ranks.items():
    print(f'DCG and Hits for {key}:')
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
      print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(value, k), k, hits_count(value, k)))

#### 1."Дефолтный токенайзер"

In [ ]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
        
tokenizer = MyTokenizer()

In [ ]:
words = words_maker(train_data, tokenizer)

In [ ]:
len(words)

1000000

Среднее количество слов в строке

In [ ]:
mean_of_words(words)

17.457746

Рандомно возьму min_count = 5, window = 5.

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                              vector_size=200,                 # embedding vector size
                              min_count=5,             # consider words that occured at least 5 times
                              window=5).wv

In [ ]:
wv_ranks_1 = wv_ranking(validation_data, embeddings_trained, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranks_1, k), k, hits_count(wv_ranks_1, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.251 | Hits@   1: 0.251
DCG@   5: 0.323 | Hits@   5: 0.389
DCG@  10: 0.348 | Hits@  10: 0.468
DCG@ 100: 0.399 | Hits@ 100: 0.719
DCG@ 500: 0.425 | Hits@ 500: 0.922
DCG@1000: 0.433 | Hits@1000: 1.000


#### 2."Дефолтный токенайзер" + lower

In [ ]:
class MyTokenizer_lower:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text.lower())
        
tokenizer = MyTokenizer_lower()

In [ ]:
words = words_maker(train_data, tokenizer)

In [ ]:
len(words)

1000000

Среднее количество слов в строке

In [ ]:
mean_of_words(words)

17.457746

Рандомно возьму min_count = 5, window = 5.

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                              vector_size=200,                 # embedding vector size
                              min_count=5,             # consider words that occured at least 5 times
                              window=5).wv

In [ ]:
wv_ranks_2 = wv_ranking(validation_data, embeddings_trained, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranks_2, k), k, hits_count(wv_ranks_2, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.312 | Hits@   1: 0.312
DCG@   5: 0.387 | Hits@   5: 0.451
DCG@  10: 0.414 | Hits@  10: 0.534
DCG@ 100: 0.466 | Hits@ 100: 0.792
DCG@ 500: 0.486 | Hits@ 500: 0.946
DCG@1000: 0.492 | Hits@1000: 1.000


#### 3.Токенайзер nltk




In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokenizer = word_tokenize

In [ ]:
words = words_maker(train_data, tokenizer)

In [ ]:
len(words)

1000000

Среднее количество слов в строке

In [ ]:
mean_of_words(words)

18.863629

Рандомно возьму min_count = 5, window = 5.

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                              vector_size=200,                 # embedding vector size
                              min_count=5,             # consider words that occured at least 5 times
                              window=5).wv

In [ ]:
wv_ranks_3 = wv_ranking(validation_data, embeddings_trained, tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranks_3, k), k, hits_count(wv_ranks_3, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.245 | Hits@   1: 0.245
DCG@   5: 0.308 | Hits@   5: 0.365
DCG@  10: 0.328 | Hits@  10: 0.428
DCG@ 100: 0.378 | Hits@ 100: 0.681
DCG@ 500: 0.408 | Hits@ 500: 0.917
DCG@1000: 0.417 | Hits@1000: 1.000


#### 4.Токенайзер nltk + lower




In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokenizer = word_tokenize

In [ ]:
words = words_maker(train_data, tokenizer, lower = True)

In [ ]:
len(words)

1000000

Среднее количество слов в строке

In [ ]:
mean_of_words(words)

18.86263

Рандомно возьму min_count = 5, window = 5.

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                              vector_size=200,                 # embedding vector size
                              min_count=5,             # consider words that occured at least 5 times
                              window=5).wv

In [ ]:
wv_ranks_4 = wv_ranking(validation_data, embeddings_trained, tokenizer, lower = True)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranks_4, k), k, hits_count(wv_ranks_4, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.288 | Hits@   1: 0.288
DCG@   5: 0.364 | Hits@   5: 0.432
DCG@  10: 0.384 | Hits@  10: 0.493
DCG@ 100: 0.435 | Hits@ 100: 0.750
DCG@ 500: 0.459 | Hits@ 500: 0.938
DCG@1000: 0.466 | Hits@1000: 1.000


#### Промежуточный вывод

Дефолтный токенайзер + lowercase работает лучше чем токенайзер из nltk, поэтому я дальше будут использовать именно его

#### 5."Дефолтный токенайзер" + lower + lemma




In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
class MyTokenizer_lower:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text.lower())
        
tokenizer = MyTokenizer_lower()

In [ ]:
words = words_maker(train_data, tokenizer, lemma = True)

In [ ]:
len(words)

1000000

Среднее количество слов в строке

In [ ]:
mean_of_words(words)

17.457746

Рандомно возьму min_count = 5, window = 5.

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                              vector_size=200,                 # embedding vector size
                              min_count=5,             # consider words that occured at least 5 times
                              window=5).wv

In [ ]:
wv_ranks_5 = wv_ranking(validation_data, embeddings_trained, tokenizer, lemma=True)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranks_5, k), k, hits_count(wv_ranks_5, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.330 | Hits@   1: 0.330
DCG@   5: 0.397 | Hits@   5: 0.459
DCG@  10: 0.421 | Hits@  10: 0.534
DCG@ 100: 0.476 | Hits@ 100: 0.797
DCG@ 500: 0.495 | Hits@ 500: 0.948
DCG@1000: 0.501 | Hits@1000: 1.000


Видно что качество еще больше выросло

#### 6."Дефолтный токенайзер" + lower + stemma




In [18]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='english')

In [22]:
class MyTokenizer_lower:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text.lower())
        
tokenizer = MyTokenizer_lower()

In [24]:
words = words_maker(train_data, tokenizer, stemma = True)

In [26]:
words[0]

['convert',
 'string',
 'to',
 'list',
 'convert',
 'googl',
 'result',
 'object',
 'pure',
 'js',
 'to',
 'python',
 'object']

In [25]:
len(words)

1000000

Среднее количество слов в строке

In [27]:
mean_of_words(words)

17.457746

Рандомно возьму min_count = 5, window = 5.

In [28]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                              vector_size=200,                 # embedding vector size
                              min_count=5,             # consider words that occured at least 5 times
                              window=5).wv

In [39]:
wv_ranks_6 = wv_ranking(validation_data, embeddings_trained, tokenizer, stemma=True)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [40]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranks_6, k), k, hits_count(wv_ranks_6, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.350 | Hits@   1: 0.350
DCG@   5: 0.422 | Hits@   5: 0.484
DCG@  10: 0.446 | Hits@  10: 0.558
DCG@ 100: 0.498 | Hits@ 100: 0.817
DCG@ 500: 0.517 | Hits@ 500: 0.957
DCG@1000: 0.521 | Hits@1000: 1.000


Качество выросло еще больше 

#### 7."Дефолтный токенайзер" + lower + stemma + stop words




In [41]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [45]:
stopWords = stopwords.words('english')

In [46]:
stopWords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [54]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='english')

In [55]:
class MyTokenizer_lower:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text.lower())
        
tokenizer = MyTokenizer_lower()

In [56]:
words = words_maker(train_data, tokenizer, stemma = True, stop_words = True)

In [57]:
words[0]

['convert',
 'googl',
 'object',
 'list',
 'pure',
 'js',
 'convert',
 'result',
 'python',
 'string']

In [58]:
len(words)

1000000

Среднее количество слов в строке

In [59]:
mean_of_words(words)

9.973258

Рандомно возьму min_count = 5, window = 5.

In [60]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                              vector_size=200,                 # embedding vector size
                              min_count=5,             # consider words that occured at least 5 times
                              window=5).wv

In [61]:
wv_ranks_7 = wv_ranking(validation_data, embeddings_trained, tokenizer, stemma=True, 
                        stop_words = True)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [63]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranks_7, k), k, hits_count(wv_ranks_7, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.395 | Hits@   1: 0.395
DCG@   5: 0.481 | Hits@   5: 0.560
DCG@  10: 0.503 | Hits@  10: 0.627
DCG@ 100: 0.551 | Hits@ 100: 0.859
DCG@ 500: 0.565 | Hits@ 500: 0.964
DCG@1000: 0.569 | Hits@1000: 1.000


Добился максимального качества среди тех идей, что озвучил в начале.

Теперь с этой отправкной точки, буду перебирать гиперпараметры W2V для достижения наилучшнего результата

#### 8."Дефолтный токенайзер" + lower + stemma + stop words + gridsearch




In [20]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
stopWords = stopwords.words('english')

In [23]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer(language='english')

In [24]:
class MyTokenizer_lower:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text.lower())
        
tokenizer = MyTokenizer_lower()

In [25]:
words = words_maker(train_data, tokenizer, stemma = True, stop_words = True)

In [26]:
words[0]

['object',
 'convert',
 'python',
 'js',
 'convert',
 'string',
 'pure',
 'list',
 'googl',
 'result']

In [27]:
len(words)

1000000

Среднее количество слов в строке

In [28]:
mean_of_words(words)

9.973258

In [29]:
vector_size = [300]
window = [3,5,7]
min_count = [3,5,7]

In [31]:
from gensim.models import Word2Vec
gs_results = gridsearch_w2v(vector_size, window, min_count)

In [33]:
gs_results

{'embs_w2v_300_3_3': <gensim.models.keyedvectors.KeyedVectors at 0x7fbe1dec3580>,
 'embs_w2v_300_3_5': <gensim.models.keyedvectors.KeyedVectors at 0x7fbe1dec3850>,
 'embs_w2v_300_3_7': <gensim.models.keyedvectors.KeyedVectors at 0x7fbe5438ec70>,
 'embs_w2v_300_5_3': <gensim.models.keyedvectors.KeyedVectors at 0x7fbdbd8f7fd0>,
 'embs_w2v_300_5_5': <gensim.models.keyedvectors.KeyedVectors at 0x7fbdbdc609a0>,
 'embs_w2v_300_5_7': <gensim.models.keyedvectors.KeyedVectors at 0x7fbdbd8dfd00>,
 'embs_w2v_300_7_3': <gensim.models.keyedvectors.KeyedVectors at 0x7fbdb8c0bc40>,
 'embs_w2v_300_7_5': <gensim.models.keyedvectors.KeyedVectors at 0x7fbdb8bb7d90>,
 'embs_w2v_300_7_7': <gensim.models.keyedvectors.KeyedVectors at 0x7fbdb8880910>}

In [44]:

with open('/content/drive/MyDrive/Colab Notebooks/DLS_Part2/01.Введение в NLP/HW_01/embeddings_trained.pkl', 'wb') as f:
    pickle.dump(gs_results, f)

In [42]:
def ranks(gs_results, validation_data, tokenizer, dim = 300, stemma=True, stop_words = True):
  results = dict()
  for key, value in gs_results.items():
    results[key] = wv_ranking(validation_data, value, tokenizer, dim = dim, stemma=stemma, stop_words = stop_words)
  return results

In [43]:
gs_ranks = ranks(gs_results, validation_data, tokenizer, stemma=True, stop_words = True)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/3760 [00:00<?, ?it/s]

In [45]:
with open('/content/drive/MyDrive/Colab Notebooks/DLS_Part2/01.Введение в NLP/HW_01/gs_ranks.pkl', 'wb') as f:
    pickle.dump(gs_ranks, f)

In [47]:
DCG_Hits_for_gridsearch(gs_ranks)

DCG and Hits for embs_w2v_300_3_3:


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.379 | Hits@   1: 0.379
DCG@   5: 0.458 | Hits@   5: 0.528
DCG@  10: 0.480 | Hits@  10: 0.594
DCG@ 100: 0.527 | Hits@ 100: 0.828
DCG@ 500: 0.543 | Hits@ 500: 0.952
DCG@1000: 0.548 | Hits@1000: 1.000
DCG and Hits for embs_w2v_300_3_5:


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.380 | Hits@   1: 0.380
DCG@   5: 0.460 | Hits@   5: 0.531
DCG@  10: 0.482 | Hits@  10: 0.597
DCG@ 100: 0.527 | Hits@ 100: 0.821
DCG@ 500: 0.544 | Hits@ 500: 0.955
DCG@1000: 0.549 | Hits@1000: 1.000
DCG and Hits for embs_w2v_300_3_7:


...

DCG@   1: 0.413 | Hits@   1: 0.413
DCG@   5: 0.495 | Hits@   5: 0.571
DCG@  10: 0.521 | Hits@  10: 0.649
DCG@ 100: 0.568 | Hits@ 100: 0.875
DCG@ 500: 0.581 | Hits@ 500: 0.973
DCG@1000: 0.584 | Hits@1000: 1.000
DCG and Hits for embs_w2v_300_7_5:


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.412 | Hits@   1: 0.412
DCG@   5: 0.496 | Hits@   5: 0.574
DCG@  10: 0.522 | Hits@  10: 0.654
DCG@ 100: 0.569 | Hits@ 100: 0.881
DCG@ 500: 0.581 | Hits@ 500: 0.974
DCG@1000: 0.584 | Hits@1000: 1.000
DCG and Hits for embs_w2v_300_7_7:


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.498 | Hits@   5: 0.574
DCG@  10: 0.524 | Hits@  10: 0.652
DCG@ 100: 0.572 | Hits@ 100: 0.881
DCG@ 500: 0.584 | Hits@ 500: 0.976
DCG@1000: 0.587 | Hits@1000: 1.000


В итоге самое лучше качество после перебора гиперпараметров получилось при window = 7, min_count = 7.

В целом можно и дальше попробовать улучшить параметры перебирая и другие гипер параметры и пробуя дальше увеличивать window и min_count. Но я остановлюсь на этом и сформулирую выводы ниже

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


 1. Я пробовал 2 токенайзера : Nltk И дефолтный (regexp). Дефолтный токенайзер давал лучшее качество, к сожаленю, так и не понял почему, я думал библиотечные методы оптимизированы и дают лучшее качество, чем простой способ токенизации ' в лоб '.

 2. Нормализациая слов улучшает качество. Причем Стемматезация работает лучше чем лемматезация (от nltk)

 3. Конечно обученные эмбеддинги на train_data справлялись лучше, ведь предобученные эмбеддинги взятые в начале обучались на другом корпусе слов, а мы под конкретную задачу обучили на своем корпусе, и поэтому получили лучшее качество, но пожертвовали временем.

 4. Не знаю какое должно быть тут идеально решение задачи, но в целом результат улучшился с DCG@1000: 0.444 до DCG@1000: 0.587 за счет применения lower() ко всем словам, стемматезации и исключения стоп слов.

 5. Возможно можно как-то использовать n-gramms для решения этой задачи. Плюс еще поиграться с гиперпараметрами алгоритма
 
